<a href="https://colab.research.google.com/github/ofSi02/githubtest/blob/main/CNRA_GWL_DWNLD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# --- Colab uploader ---
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()  # choose WellsForDEMO.csv (or your file)

# Grab the first uploaded file
uploaded_name = next(iter(uploaded))
print("Uploaded:", uploaded_name)

# Read it into a DataFrame
wells_df = pd.read_csv(io.BytesIO(uploaded[uploaded_name]))
print("Wells file shape:", wells_df.shape)
print("Columns:", list(wells_df.columns))

# ----------------------------
# CONFIG (EDIT IF NEEDED)
# ----------------------------
RESOURCE_ID = "bfa9f262-24a1-45bd-8dc8-138bc8107266"
OUTPUT_CSV  = "GWData_filtered.csv"

# Column in your wells file that contains site codes.
# If your file uses a different name, change it here.
WELLS_SITE_COL = "site_code"   # e.g., "SITE_CODE" also fine, script is case-insensitive

CHUNK_SIZE = 40
MAX_ROWS_PER_QUERY = 100000

BASE_URL_SQL    = "https://data.cnra.ca.gov/api/3/action/datastore_search_sql"
BASE_URL_SEARCH = "https://data.cnra.ca.gov/api/3/action/datastore_search"


Saving WellsForDEMO.csv to WellsForDEMO.csv
Uploaded: WellsForDEMO.csv
Wells file shape: (43, 16)
Columns: ['SITE_CODE', 'WELL_NAME', 'SWN', 'Local_Well_Name', 'County', 'STATION_ORGANIZATION', 'USE', 'COMPLETION', 'LAT', 'LONG', 'STN_CMT', 'RPE_FT', 'GSE_FT', 'WELL_DEPTH_FT', 'GSE_MO', 'GSE_MT']


In [4]:
import math
import requests

# ---- Step 1: Verify resource & fields ----
check = requests.get(BASE_URL_SEARCH, params={"resource_id": RESOURCE_ID, "limit": 1})
check.raise_for_status()
meta = check.json()

if not meta.get("success", False):
    raise RuntimeError(f"Resource check failed: {meta.get('error') or meta}")

fields = [f["id"] for f in meta["result"]["fields"]]
print("Available fields in resource:", fields)

# The CNRA table field is 'site_code' (lowercase) for this dataset
SITE_FIELD = "site_code"
if SITE_FIELD not in fields:
    raise RuntimeError(f"Expected '{SITE_FIELD}' not found. Pick from: {fields}")

# ---- Step 2: Find site_code column in uploaded wells file (case-insensitive) ----
col_map = {c.strip().lower(): c for c in wells_df.columns}
wanted = WELLS_SITE_COL.strip().lower()

if wanted not in col_map:
    raise RuntimeError(
        f"Could not find column '{WELLS_SITE_COL}' in your wells file.\n"
        f"Available columns: {list(wells_df.columns)}"
    )

site_col = col_map[wanted]

# Extract unique site codes, normalized to lowercase for matching
site_codes = (
    wells_df[site_col]
    .dropna()
    .astype(str)
    .str.strip()
)
site_codes = [s for s in site_codes if s != ""]
site_codes_lc = sorted(set(s.lower() for s in site_codes))

print(f"Found {len(site_codes_lc)} unique site codes (case-insensitive)")

# ---- Step 3: Fetch one chunk (case-insensitive SQL match) ----
def fetch_chunk(chunk_codes_lc):
    if not chunk_codes_lc:
        return []

    # Escape single quotes, then build SQL IN list
    in_list = ", ".join("'" + c.replace("'", "''") + "'" for c in chunk_codes_lc)

    # Case-insensitive match: LOWER(site_code) IN (...)
    sql = f"""
        SELECT *
        FROM "{RESOURCE_ID}"
        WHERE LOWER("{SITE_FIELD}") IN ({in_list})
        ORDER BY LOWER("{SITE_FIELD}"), msmt_date
        LIMIT {MAX_ROWS_PER_QUERY}
    """

    r = requests.get(BASE_URL_SQL, params={"sql": sql})
    r.raise_for_status()

    try:
        payload = r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON response: {r.status_code} {r.text[:400]}")

    if not payload.get("success", False):
        raise RuntimeError(f"CKAN error: {payload.get('error') or payload}")

    return payload["result"]["records"]

# ---- Step 4: Loop through chunks ----
all_records = []
n_chunks = math.ceil(len(site_codes_lc) / CHUNK_SIZE)

for i in range(0, len(site_codes_lc), CHUNK_SIZE):
    chunk = site_codes_lc[i:i + CHUNK_SIZE]
    print(f"Fetching chunk {i//CHUNK_SIZE + 1} of {n_chunks} ({len(chunk)} codes)...")
    records = fetch_chunk(chunk)
    print(f"  -> got {len(records)} rows")
    all_records.extend(records)

# ---- Step 5: Save + download ----
df = pd.DataFrame(all_records)
df.to_csv(OUTPUT_CSV, index=False)

print(f"Done. Wrote {len(df)} rows to {OUTPUT_CSV}")
print("Preview:")
display(df.head())

# Trigger download in Colab
files.download(OUTPUT_CSV)


Available fields in resource: ['_id', 'site_code', 'msmt_date', 'wlm_rpe', 'wlm_gse', 'gwe', 'gse_gwe', 'wlm_qa_desc', 'wlm_qa_detail', 'wlm_mthd_desc', 'wlm_acc_desc', 'wlm_org_name', 'coop_org_name', 'monitoring_program', 'basin_code', 'county_name', 'well_use', 'source', 'msmt_cmt']
Found 43 unique site codes (case-insensitive)
Fetching chunk 1 of 2 (40 codes)...
  -> got 23345 rows
Fetching chunk 2 of 2 (3 codes)...
  -> got 3459 rows
Done. Wrote 26804 rows to GWData_filtered.csv
Preview:


,_id,_full_text,site_code,msmt_date,wlm_rpe,wlm_gse,gwe,gse_gwe,wlm_qa_desc,wlm_qa_detail,wlm_mthd_desc,wlm_acc_desc,wlm_org_name,coop_org_name,monitoring_program,basin_code,county_name,well_use,source,msmt_cmt
0,3406705,"'-004':25 '-10':3 '-25':4 '00':5,6,7 '1981':2 ...",410413N1211147W001,1981-10-25T00:00:00,4128.400,4127.400,4088.900,38.500,Good,None,Unknown,Unknown,Department of Water Resources,Department of Water Resources,SGMA,5-004,Lassen,Irrigation,DWR_DISCRETE,None
1,3406704,"'-004':22 '-10':3 '-26':4 '00':5,6,7 '10/13/19...",410413N1211147W001,1981-10-26T00:00:00,4127.900,4127.400,None,None,Missing,Special/Other,None,None,Department of Water Resources,Department of Water Resources,SGMA,5-004,Lassen,Irrigation,DWR_DISCRETE,Not measured again until 10/13/1983.
2,3406703,"'-004':25 '-10':3 '-13':4 '00':5,6,7 '1983':2 ...",410413N1211147W001,1983-10-13T00:00:00,4128.400,4127.400,4098.600,28.800,Good,None,Unknown,Unknown,Department of Water Resources,Department of Water Resources,SGMA,5-004,Lassen,Irrigation,DWR_DISCRETE,None
3,3406702,"'-004':25 '-04':3 '-12':4 '00':5,6,7 '17.700':...",410413N1211147W001,1984-04-12T00:00:00,4127.900,4127.400,4109.700,17.700,Good,None,Unknown,Unknown,Department of Water Resources,Department of Water Resources,SGMA,5-004,Lassen,Irrigation,DWR_DISCRETE,None
4,3406701,"'-004':25 '-01':4 '-11':3 '00':5,6,7 '1984':2 ...",410413N1211147W001,1984-11-01T00:00:00,4127.900,4127.400,4099.500,27.900,Good,None,Unknown,Unknown,Department of Water Resources,Department of Water Resources,SGMA,5-004,Lassen,Irrigation,DWR_DISCRETE,None


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>